!pip install torcheeg #suppose having pytorch

In [5]:
#class declration
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder

class capilabDS(Dataset):
    def __init__(self, x, y, device = 'cpu'):
        self.x = x
        self.y = y
        self.device = device
        self.num_class = np.unique(self.y)
        ohe = OneHotEncoder(sparse_output = False)
        self.y = ohe.fit_transform(self.y, self.num_class)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        
        x = torch.Tensor(self.x[index,:,:,:]).to(self.device, torch.float32)
        y = torch.Tensor(self.y[index, :]).to(self.device, torch.float32)

        return (x, y)

## Inference

In [7]:
import torch
from torcheeg.models import EEGNet
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cuda':
    model = torch.load('rose2023bmi').to(device)
else:
    model = torch.load('rose2023bmi', map_location='cpu') # in case no cuda support


xv = np.load('x_test.npy')
yv = np.load("y_test.npy")
test_data = capilabDS(xv, yv, device = device)
test_loader = DataLoader(test_data, batch_size=64, shuffle = False)
bacc = 0
with torch.no_grad():
    for i, batch in enumerate(test_loader):
        x, y = batch
        yhat = model(x)

        pred = np.argmax(nn.functional.softmax(yhat, 1).detach().cpu().numpy(), 1)
        target = np.argmax(y.detach().cpu().numpy(), 1)
        bacc += np.sum(pred == target)
        print(target, pred)
    print(f"accuracy {bacc / len(test_loader.dataset) * 100}%")


[0 1 0 2 1 2 1 0 2 1 1 1 0 0 2 2 1 2 2 1 0 2 0] [2 1 1 0 0 1 0 0 0 1 2 1 0 2 1 2 1 1 2 0 0 2 1]
accuracy 43.47826086956522%
